In [1]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_pendulum import get_pendulum_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


2024-06-02 09:51:59.030697: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 09:52:03.014145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Generate data

In [5]:
training_data = get_pendulum_data(100, True)
validation_data = get_pendulum_data(10)

Generating pendulum data with noise
Generating pendulum data


# Set up model and training parameters

In [6]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 1
params['model_order'] = 2
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(2*params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-4
params['loss_weight_sindy_z'] = 5e-5
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [128,64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1000
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [9]:
num_experiments = 10
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'pendulum_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING


2024-05-27 22:25:41.939832: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 22:25:41.939951: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 22:25:41.940004: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 22:25:42.175167: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 22:25:42.175283: I external/local_xla/xla/stream_executor

Epoch 0
   training loss 0.01818026974797249, (0.008089201, 3.3523893, 19.826984, 0.9957164)
   validation loss 0.01711106486618519, (0.008153872, 3.365014, 17.557968, 0.9957164)


2024-05-27 22:25:48.072352: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 52020000 exceeds 10% of free system memory.
2024-05-27 22:25:48.091250: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 52020000 exceeds 10% of free system memory.


decoder loss ratio: 0.972113, decoder SINDy loss  ratio: 1.000000


2024-05-27 22:30:10.283258: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 52020000 exceeds 10% of free system memory.


Epoch 100
   training loss 0.017329752445220947, (0.007409425, 2.6976362e-05, 19.826986, 0.6832834)
   validation loss 0.016248520463705063, (0.007462703, 2.8477038e-05, 17.557966, 0.6832834)
decoder loss ratio: 0.889712, decoder SINDy loss  ratio: 1.000000
Epoch 200
   training loss 0.017307966947555542, (0.007391831, 0.00075919024, 19.826986, 0.26047003)
   validation loss 0.01623866707086563, (0.007457037, 0.0008502341, 17.557966, 0.26047003)
decoder loss ratio: 0.889036, decoder SINDy loss  ratio: 1.000000
